In [1]:
import sqlite3
import pandas as pd
  
# Connect to SQLite database
conn = sqlite3.connect(r'./db/vivino.db')
  
# Create a cursor object
cur = conn.cursor()

In [2]:
q = """
SELECT name
FROM sqlite_schema
WHERE type = 'table' AND name NOT LIKE 'sqlite_%';
"""

cur.execute(q).fetchall()

[('countries',),
 ('grapes',),
 ('wineries',),
 ('flavor_groups',),
 ('keywords',),
 ('regions',),
 ('most_used_grapes_per_country',),
 ('toplists',),
 ('wines',),
 ('vintages',),
 ('keywords_wine',),
 ('vintage_toplists_rankings',)]

# We want to highlight 10 wines to increase our sales. Which ones should we choose and why?

In [3]:
q = """
SELECT name, price_euros, ratings_average
FROM (
SELECT *
FROM vintages
JOIN wines
ON wines.id = vintages.wine_id
ORDER BY ratings_average DESC
LIMIT 10);
"""

cur.execute(q).fetchall()
df  = pd.read_sql_query (q, conn)
df

,name,price_euros,ratings_average
0,Château Doisy-Daëne L'Extravagant de Doisy-Daë...,397.50,4.9
1,Biondi-Santi Brunello di Montalcino Riserva 1955,1970.05,4.9
2,Château Lafite Rothschild Pauillac (Premier Gr...,1900.00,4.8
3,Krug Clos du Mesnil 1998,3522.50,4.8
4,Salon Le Mesnil Blanc de Blancs (Cuvée S) Brut...,2882.50,4.8
5,Dal Forno Romano Amarone della Valpolicella Mo...,808.75,4.8
6,Harlan Estate Harlan Estate Red 1998,1921.25,4.8
7,Krug Clos du Mesnil 2000,3096.25,4.8
8,Quinta do Noval Vintage Port Nacional 2001,1152.50,4.8
9,Pétrus Pomerol 1990,6300.00,4.8


In [4]:
q = """
SELECT name, price_euros, user_structure_count
FROM (
SELECT *
FROM vintages
JOIN wines
ON wines.id = vintages.wine_id
ORDER BY user_structure_count DESC
LIMIT 10);
"""

cur.execute(q).fetchall()
df  = pd.read_sql_query (q, conn)
df

,name,price_euros,user_structure_count
0,Dom Pérignon Brut Champagne 2000,638.83,13491
1,Dom Pérignon Brut Champagne 2004,460.00,13491
2,Dom Pérignon Brut Champagne 2006,440.00,13491
3,Dom Pérignon Brut Champagne 2013,250.00,13491
4,Dom Pérignon Brut Champagne 2012,397.50,13491
5,Antinori Tignanello 2005,750.00,8219
6,Antinori Tignanello 1997,491.25,8219
7,Antinori Tignanello 2003,840.00,8219
8,Antinori Tignanello 1989,287.50,8219
9,Antinori Tignanello 1998,820.00,8219


In [5]:
q = """
WITH q AS (
    SELECT *
    FROM toplists
    JOIN countries ON toplists.country_code = countries.code
),
p AS (
    SELECT *
    FROM vintages
    JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
)
    SELECT DISTINCT p.name, p.price_euros, q.name, p.wine_id
    FROM q
    JOIN p ON p.top_list_id = q.id
    ORDER BY q.users_count DESC
    LIMIT 10;

"""

cur.execute(q).fetchall()
df  = pd.read_sql_query (q, conn)

df

,name,price_euros,name,wine_id
0,Gitana Lupi Rezerva 2017,36.55,Bestsellers in Arizona,1468452
1,Dom Pérignon Brut Champagne 2013,250.00,Bestsellers in Tennessee,86684
2,Billecart-Salmon Brut Rosé Champagne,81.50,Bestsellers in Arizona,1211816
3,Buena Vista Chateau Buena Vista Cabernet Sauvi...,61.95,Bestsellers in Utah,5261
4,Zenato Amarone della Valpolicella Classico 2017,51.90,Bestsellers in Nevada,12393
5,San Marzano 60 Sessantanni Old Vines Primitivo...,24.75,Bestsellers in Massachusetts,11890
6,San Marzano 60 Sessantanni Old Vines Primitivo...,24.75,Bestsellers in North Dakota,11890
7,Chateau Montelena Chardonnay 2019,78.95,Bestsellers in Montana,1697
8,Château Pape Clément Pessac-Léognan (Grand Cru...,121.95,Bestsellers in New Mexico,82613
9,Kistler Les Noisetiers 2020,94.00,Bestsellers in Wyoming,1155762


# We have a limited marketing budget for this year. Which country should we prioritise and why?


In [6]:
q = """ 
SELECT name, users_count
FROM countries
ORDER by users_count DESC
LIMIT 1
"""

cur.execute(q).fetchall()

df  = pd.read_sql_query (q, conn)
df


,name,users_count
0,États-Unis,12273684


# We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why?


## Most wines produced

In [7]:
q = """ 
WITH wineries_ids AS (
SELECT name AS wine_name, winery_id, count(*) AS num_wines, id AS wine_id
FROM wines
GROUP BY winery_id)

SELECT SUBSTR(REPLACE(vintages.name, wineries_ids.wine_name, ''), 1, LENGTH(REPLACE(vintages.name, wineries_ids.wine_name, ''))-5) as winery_name, 
wineries_ids.wine_name, wineries_ids.winery_id, wineries_ids.num_wines
FROM vintages
JOIN wineries_ids ON wineries_ids.wine_id = vintages.wine_id
GROUP BY winery_name
ORDER BY num_wines DESC
LIMIT 3



"""
cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df



#These winery_id don't correspond with any ids in wineries table so we can't have the name of these wineries

,winery_name,wine_name,winery_id,num_wines
0,Gaja,Barbaresco,10474,12
1,Domaine Faiveley,Nuits-Saint-Georges 1er Cru Les Damodes,9294,10
2,Antinori,Tenuta Tignanello 'Solaia',1252,9


## Quantity of the bestsellers produced


In [8]:
rq = """
WITH q AS (
    SELECT *
    FROM toplists
    JOIN countries ON toplists.country_code = countries.code
),
p AS (
    SELECT *
    FROM vintages
    JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
),
w as(
    SELECT DISTINCT p.name as wine_name, p.price_euros, q.name, p.wine_id
    FROM q
    JOIN p ON p.top_list_id = q.id
), wineries_ids AS(
SELECT w.wine_name, winery_id, count(*) AS num_wines, w.wine_id
FROM wines
JOIN w ON w.wine_id = wines.id
GROUP BY winery_id)

SELECT SUBSTR(REPLACE(vintages.name, wineries_ids.wine_name, ''), 1, LENGTH(REPLACE(vintages.name, wineries_ids.wine_name, ''))-5) as winery_name, 
wineries_ids.wine_name, wineries_ids.winery_id, wineries_ids.num_wines
FROM vintages
JOIN wineries_ids ON wineries_ids.wine_id = vintages.wine_id
GROUP BY winery_name
ORDER BY num_wines DESC
LIMIT 3

"""

cur.execute(rq).fetchall()
df  = pd.read_sql_query (rq, conn)
df

#these wineryids also doesn't appear in wineries table 


,winery_name,wine_name,winery_id,num_wines
0,San Marzano 60 Sessantanni Old Vines Primitivo...,San Marzano 60 Sessantanni Old Vines Primitivo...,55648,5
1,Zenato Amarone della Valpolicella Classico,Zenato Amarone della Valpolicella Classico 2017,1583,1
2,Pintia Toro,Pintia Toro 2018,1740,1


## Highest average rating

In [9]:
q = """ 

SELECT winery_id, AVG(ratings_average) AS avg_rating
FROM wines
GROUP BY winery_id
ORDER BY avg_rating DESC
LIMIT 3;


"""
cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df


,winery_id,avg_rating
0,277785,4.8
1,266660,4.8
2,14919,4.8


# We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus (note that these keywords are case sensitive ⚠️). We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

In [12]:

q = """ 
WITH flavour_ids AS (
SELECT id, name
FROM keywords
WHERE name IN ('coffee', 'toast', 'green', 'apple', 'cream', 'citrus'))

SELECT DISTINCT wine_id, count, wines.name, flavour_ids.id, flavour_ids.name
FROM keywords_wine
JOIN flavour_ids ON keywords_wine.keyword_id = flavour_ids.id
JOIN wines ON wines.id = keywords_wine.wine_id
WHERE count >= 10
"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df


,wine_id,count,name,id,name
0,82698,15,Brunello di Montalcino Riserva,117,coffee
1,1166837,49,Pomerol,117,coffee
2,1166837,18,Pomerol,123,cream
3,1127349,240,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champ...,106,citrus
4,1127349,100,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champ...,417,toast
...,...,...,...,...,...
1067,74300,125,Blanc de Blancs Brut Champagne (Vintage),9,apple
1068,74300,143,Blanc de Blancs Brut Champagne (Vintage),106,citrus
1069,74300,56,Blanc de Blancs Brut Champagne (Vintage),417,toast
1070,74300,92,Blanc de Blancs Brut Champagne (Vintage),123,cream


# We would like to select wines that are easy to find all over the world. Find the top 3 most common grapes all over the world and for each grape, give us the the 5 best rated wines.

In [13]:
q = """ 
WITH grape_ids AS (
SELECT grape_id, SUM(wines_count) as total_wine, country_code
FROM most_used_grapes_per_country
GROUP BY grape_id
ORDER BY total_wine DESC
LIMIT 3),
grape_name AS(
SELECT name, id, country_code
FROM grapes
JOIN grape_ids ON grape_ids.grape_id = grapes.id),
rating AS (
SELECT wines.name, grape_name.name AS grape_name, wines.ratings_average 
FROM wines 
JOIN regions ON wines.region_id = regions.id
JOIN grape_name ON grape_name.country_code = regions.country_code
ORDER BY wines.ratings_average DESC),
cabernet AS (
SELECT *
FROM rating 
WHERE grape_name = 'Cabernet Sauvignon'
LIMIT 5),
merlot AS (
SELECT *
FROM rating 
WHERE grape_name = 'Merlot'
LIMIT 5)

SELECT *
FROM rating 
WHERE grape_name = 'Chardonnay'
LIMIT 5

"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df

,name,grape_name,ratings_average
0,Amarone della Valpolicella Classico Riserva,Chardonnay,4.8
1,Fratini Bolgheri Superiore,Chardonnay,4.8
2,Cerretalto Brunello di Montalcino,Chardonnay,4.7
3,Barolo Riserva Monfortino,Chardonnay,4.7
4,Amarone della Valpolicella Classico,Chardonnay,4.7


# We would like to create a country leaderboard. Come up with a visual that shows the average wine rating for each country. Do the same for the vintages.

In [14]:
q = """ 
SELECT name, wines_count
FROM countries

"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)

import plotly.express as px
fig = px.line(df, x='name', y="wines_count",
               labels={
                     "name": "Countries",
                     "wines_count": "Wines Count"})
fig.show()

In [15]:
q = """ 
SELECT region_id, AVG(ratings_average) as average_rating, countries.name
FROM wines
JOIN regions ON regions.id = wines.region_id
JOIN countries ON regions.country_code = countries.code
GROUP BY countries.name


"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df
import plotly.express as px
fig = px.line(df, x='name', y="average_rating",
               labels={
                     "name": "Countries",
                     "wines_count": "Wines Rating"})
fig.show()

# One of our VIP clients likes _Cabernet Sauvignon_ and would like our top 5 recommendations. Which wines would you recommend to him?


In [16]:
q = """ 
WITH w AS (
    SELECT
        name,
        AVG(acidity) AS avg_acidity,
        AVG(fizziness) AS avg_fizziness,
        AVG(intensity) AS avg_intensity,
        AVG(sweetness) AS avg_sweetness,
        AVG(tannin) AS avg_tannin
    FROM wines
    WHERE acidity IS NOT NULL AND intensity IS NOT NULL 
          AND sweetness IS NOT NULL AND tannin IS NOT NULL
    GROUP BY name
), 
a AS (
    SELECT         
        AVG(acidity) AS avg_acidity,
        AVG(fizziness) AS avg_fizziness,
        AVG(intensity) AS avg_intensity,
        AVG(sweetness) AS avg_sweetness,
        AVG(tannin) AS avg_tannin
    FROM wines
    WHERE name = 'Cabernet Sauvignon'
)
SELECT w.name,
       w.avg_acidity,
       w.avg_fizziness,
       w.avg_intensity,
       w.avg_sweetness,
       w.avg_tannin,
       ABS(w.avg_acidity - a.avg_acidity) as acidity_difference,
       ABS(w.avg_intensity - a.avg_intensity) as intensity_difference,
       ABS(w.avg_sweetness - a.avg_sweetness) as sweetness_difference,
       ABS(w.avg_tannin - a.avg_tannin) as tannin_difference,
       (ABS(w.avg_acidity - a.avg_acidity) *  ABS(w.avg_intensity - a.avg_intensity) * ABS(w.avg_sweetness - a.avg_sweetness) * ABS(w.avg_tannin - a.avg_tannin)) AS combined_difference
       
FROM w
JOIN a
WHERE name NOT LIKE '%Cabernet Sauvignon%' 
ORDER BY combined_difference
LIMIT 3

"""

cur.execute(q).fetchall()
df = pd.read_sql_query(q, conn)
df


,name,avg_acidity,avg_fizziness,avg_intensity,avg_sweetness,avg_tannin,acidity_difference,intensity_difference,sweetness_difference,tannin_difference,combined_difference
0,Altura,3.088019,None,4.335320,1.750534,3.434145,0.223193,0.060686,0.002190,0.015943,4.729554e-07
1,Montelìg,3.112812,None,4.594468,1.748861,3.491792,0.198399,0.198462,0.000517,0.041703,8.490276e-07
2,Victorino,2.778029,None,4.398544,1.834238,3.422341,0.533183,0.002538,0.085894,0.027747,3.224923e-06
